<a href="https://colab.research.google.com/github/5starkarma/face_smoothing/blob/master/face_smoothing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone https://github.com/5starkarma/face-smoothing.git

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
% cd /content/drive/My\ Drive/Colab Notebooks/face-smoothing

/content/drive/My Drive/Colab Notebooks/face-smoothing


In [3]:
pip install -U PyYAML

     |████████████████████████████████| 276kB 2.8MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44619 sha256=a269c132079e5252c33cfb4feb1627665a6c67e1dee81236c1739f0bd6337a2d
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built PyYAML
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
! python3 infer.py --save-steps

Face detected: [113, 152, 464, 612]
Operation ran in 0.2764 seconds


In [ ]:
! rm -r /content/drive/My\ Drive/Colab\ Notebooks/face-smoothing/data/output